# <center>  Data Inversion for Aerosol Science and Technology </center> 

__Author:__ Markus Petters (mdpetter@ncsu.edu)

__Abstract__. Many aerosol measurement techniques produce raw measurement response functions that must be inverted to properly interpret the data. This tutorial will introduce common inversion approaches used in Aerosol Science & Technology. One often used technique is mobility classification of aerosol using electrical mobility analyzers. The tutorial will briefly introduce the aerosol sampling with mobility analyzers and provide hands-on examples for several instrument configurations. The examples are designed to demonstrate how the inversion of mobility analyzer response functions is critical to informing experimental design and data analysis. At the end of the session, tutorial participants will have a starting point to modify supplied computer code for use in their own research projects.

<div class="alert alert-success">
 
**By the end of this tutorial you will be able to**

- Create and manipulate size distributions in code.
- Represent cylindrical and radial DMAs with arbitrary dimensions in code.
- Review the theory governing DMA transmission.
- Compute the moments of output from a DMA operated at a single voltage.
- Compute the convolution matrix for DMAs with corresponding to a custom instrument.
- Apply the convolution matrix to compute the response function from a size distribution.
- Evaluate how measurement noise influences the least-squares inversion scheme.
- Apply regularization to smooth the least-squares inversion results.
- Apply the L-curve to identify the optimal regularization parameter.
- Apply the code to invert a DMA response function loaded from a data file.
- Apply the code to invert size resolved CCN data loaded from a data file.

</div>


<div class="alert alert-danger">

**Instructions**
    
- Go to Menu "Cell" -> "Run All Below"
- The notebook will initialize all cells (~30 s)
- Navigate back to this point and start with the introduction video.    
- Work through the notebook sequentially.
- When finished, move to Session 2.
 
</div>

# Welcome

In [ ]:
include("Scripts/startup.jl")
display("https://www.youtube.com/watch?v=9SDaLLM7vrg")

## Navigation and Symbols

In [ ]:
play("Audio/colored-boxes.ogg")

<div class="alert alert-success">

**You met the following learning objective**

- Create and delete cells in Jupyter.

</div>

<div class="alert alert-warning">

**This is a key point**
    
Newton's law will do the trick: $F = ma$

</div>

<div class="alert alert-danger">

**Important additional information**
    
- You need some calculus to find the velocity from $F = ma$.
- Further details are in the Feynman Lectures on Physics.

</div>

<div class="alert alert-info">

**Assignment**

Calculate the force a 1 kg heavy ball will exert on a scale.

</div>


## Jupyter Cells

In [ ]:
play("Audio/environment.ogg")

In [ ]:
println("Hello World")

<div class="alert alert-info">
Write one line of code to print "Aerosol Science Rocks!".
</div>

<div class="alert alert-info">
Create a new cell below this one calculate 1+1
</div>

# 1. Introduction

## 1.1 Inverse Problems

In [ ]:
play("Audio/introduction.ogg")

<img src="Figures/inverse.png" width="400" align="left">

<br clear="all" />

<div class="alert alert-warning">

**Approach to solve the inverse problem**

1. Write Fredholm integral equations of the first kind for specific problem
2. Discretize equation to write in Matrix form
3. Apply inversion with regularization with constraints
    
</div>



## 1.2 The Differential Mobility Analyzer

In [ ]:
play("Audio/dma.ogg")

<img src="Figures/nbs1_f01.png" width="400"> 
<b> Figure. </b> Schematic of a differential mobility analyzer. The cylindrical differential mobility analyzer is an annular capacitor. The column's properties are defined by the radii $r_1$, $r_2$, the length of the aerosol path, $l$. Operation conditions are defined by the the electric potential $v$ applied across the annulus and the four flow rates: sheath flow, $q_{sh}$, polydisperse aerosol flow $q_a$, excess flow, $q_{ex}$, and monodisperse sample flow, $q_{sa}$. Throughout this work it is assumed that the flows are balanced, i.e. $q_{sh} = q_{ex}$ and $q_{sa} = q_a$. The two flows tracked are $q_{sh}$ and $q_{sa}$. Dimensions and flow range for one commonly used model are provided in Table 1 below. <br>


<center>
<table style="width:45%">
    <caption> <b> Table. </b> Geometry of the TSI 3080 long cylindrical DMA column together with typical ranges for operation parameters. </caption>
  <tr>
    <th align="justify">Geometry parameter</th>
    <th align="justify">Value</th> 
    <th align="justify">Operational parameter</th>
    <th align="justify">Range</th> 
</tr>
  <tr>
    <td align="justify">$r_1$</td>
    <td align="justify">9.37×10⁻³ m</td> 
    <td align="justify">$v$</td>
    <td align="justify">± 10-10000 V</td> 
</tr>
  <tr>
    <td align="justify">$r_2$</td>
    <td align="justify">1.961×10⁻² m</td> 
    <td align="justify">$q_{sh}$</td>
    <td align="justify">2-20 L min⁻¹</td> 
</tr>
  <tr>
    <td align="justify">$l$</td>
    <td align="justify">0.44369 m</td> 
    <td align="justify">$q_{sa}$</td>
    <td align="justify">0.3-2 L min⁻¹</td> 
  </tr>
</table>
</center>


# 2. Size Distribution

## 2.1 Histogram Representation 

In [ ]:
play("Audio/size-distribution1.ogg")

In [ ]:
include("Scripts/size_distribution1.jl")

<b> Table. </b> Example representation of a lognormal aerosol size distribution on a 10-bin geometrically stepped grid ranging from 30 to 300 nm with geometric standard deviation 1.2 and total number concentration 200 cm⁻³ . Column 1 gives the bin number, Dlow is the lower bin diameter, Dup is the upper bin diameter, ΔlnD = ln(Dup) - ln(Dlow), Dp = $\sqrt{Dup \cdot Dlow}$ is the geometric midpoint diameter of the bin, S $\equiv \frac{dN}{d\ln Dp}$ is the number spectral density [cm⁻³] and N = $\frac{dN}{d\ln Dp} \cdot d\ln Dp$ is the number concentration in each bin [cm⁻³]. The total number concentration is recovered by summing over all bins, i.e. Nt = $\sum_{i=1}^{10} N$. For a geometrically stepped grid, the ΔlnD is identical in each bin. Numbers are rounded for clarity. 

#### Size Distribution Function

The multi-modal lognormal size distribution is given by (e.g. Seinfeld and Pandis, 2006)
<center> $\frac{dN}{d\ln D_p} = \sum_{i=1}^n \frac{N_{t,i}}{\sqrt{2\pi}\ln\sigma_{g,i}} \exp \left(- \frac{\left[\ln D_p-\ln D_{pg,i}\right]^2}{2\ln \sigma_{g,i}^2}\right) $  </center>
where $\frac{dN}{d\ln D_p}$ is the spectral number density, $N_{t,i}$ is the total number concentration, $\sigma_{g,i}$ is the geometric standard deviation, and $D_{pg,i}$ is the geometric mean diameter of the $i^{th}$ mode, and $n$ is the number of modes. <br> 

## 2.2 Representing the Size Distribution in Code

In [ ]:
play("Audio/size-distribution2.ogg")

The size distribution is a composite data type (structure) that has the fields Dp, De, ΔlnD, S, and N, corresponding to the table representation above.

<div class="alert alert-warning">
    
- 𝕟.Dp: Geometric midpoint diameters
- 𝕟.De: Bin edge diameters
- 𝕟.ΔlnD: Log bin spacing, ΔlnD = ln(Dup/Dlow)
- 𝕟.S: Spectral density
- 𝕟.N: Number concentration in the bin

</div>

<div class="alert alert-danger">
    
**Blackboard bold font**
    
𝕒, 𝕓, 𝕔, 𝕕, 𝕖, 𝕗, 𝕘
</div>

In [ ]:
# This creates a size distribution with 10 bins between 30 and 300 nm. 
# It is the same size distribution as shown in Table 1.
𝕟 = lognormal([[100, 80, 1.2]]; d1 = 30.0, d2 = 300.0, bins = 10);

In [ ]:
# This is the array of bin edge diameters. Compare to Dlow and Dup in Table 1.
𝕟.De

<div class="alert alert-info">
Write one line of code to display the number concentration in each bin.
</div>

<div class="alert alert-info">
Write one line of code to display the midpoint diameters.
</div>

<div class="alert alert-info">
Write one line of code to extract the number concentration in the 5th bin. Julia starts indexing at 1. Square brackets are used to extract arrays, i.e. x[2] extracts the second element of array x.
</div>

<div class="alert alert-info">
Write one line of code to create a size distribution from 10 to 1000 nm with 256 size bins. Call this size distribution myN.
</div>

## 2.3 Size Distribution Arithmetic

In [ ]:
play("Audio/size-distribution3.ogg")

### 2.3.1 Addition of two size distributions
Let 𝕟₁ and 𝕟₂ denote a two size distribution defined on the same diameter grid. Then
```julia
𝕩 = 𝕟₁ + 𝕟₂ 
```
is defined such that 
```julia
𝕩.S = 𝕟₁.S + 𝕟₂.S 
𝕩.N = 𝕩.S .* 𝕟.ΔlnD 
```

<div class="alert alert-warning">
The net result is the superposition of size distributions.
</div>

<div class="alert alert-danger">
The bins don't have be identical. The summation will rebin 𝕟₂ onto grid 𝕟₁.
</div>

In [ ]:
# Example addition of size distributions
𝕟₁ = lognormal([[120, 90, 1.20]]; d1 = 10.0, d2 = 1000.0, bins = 256);  # size distribution
𝕟₂ = lognormal([[90, 140, 1.15]]; d1 = 20.0, d2 = 800.0, bins = 256);    # size distribution
𝕩 = 𝕟₁ + 𝕟₂
include("Scripts/plot_sizedistribution1.jl")

<b> Figure. </b> Illustration of the operation 𝕟₁+𝕟₂. The operation intuitively superimposed the two distributions into a single distribution. 

### 2.3.2 Multiplication of vector and size distribution
Let T denote a 1D vector that has the same number of elements as the size distribution $𝕟$. Then
```julia
𝕩 = T * 𝕟 
```
is defined such that 
```julia
𝕩.N = T * 𝕟.N
𝕩.S = T * 𝕟.S
```


<div class="alert alert-warning">
The net result is a bin-by-bin scaling of the number concentration.                               
</div>


In [ ]:
𝕟 = lognormal([[100, 100, 1.4]]; d1 = 10.0, d2 = 1000.0, bins = 128)  
μ,σ = 100.0, 200.0
T(x) = @. 0.5*(1.0 + erf((x - μ)/(sqrt(2σ)))) # Simple error function with mean μ and std. dev σ
𝕩 = T(𝕟.Dp)*𝕟    
include("Scripts/plot_sizedistribution2.jl")

<b> Figure. </b> Illustration of the operation T$*$𝕟. The array T is defined for each diameter point of the distribution. The convolution 𝕩 = T$*$𝕟 also applies to number concentration (not shown).

### 2.3.3 Multiplication of a scalar and a size distribution

Let a denote a floating point scalar and $𝕟$ denote a size distribution. Then
```julia
𝕩 = a⋅𝕟
```
is defined such that 
```julia
𝕩.Dp = a*𝕟.Dp 
```

<div class="alert alert-warning">
The net result is a uniform diameter shift of the size distribution.
</div>


In [ ]:
a = 2.0 # Note that a must be a floating point number
𝕟 = lognormal([[300, 100, 1.3]]; d1 = 10.0, d2 = 1000.0, bins = 256);  # size distribution
𝕩 = a⋅𝕟 
include("Scripts/plot_sizedistribution3.jl")

<b> Figure. </b> Illustration of the operation a⋅𝕟. The operation shifts the size distribution by a factor of two. Note, that the underlying diameter grid remains unchanged. This is achieved by interpolating the shifted size distribution back onto the original diameter grid. For this reason, the final fields dp and de are unaltered. 

### 2.3.4 Multiplication of a matrix and a size distribution
Let <b>A</b> denote a $n\times n$ matrix where $n$ equals the number of size bins of $𝕟$. Then
```julia
𝕩 = 𝐀*𝕟 
```
is defined such that 
```julia
𝕩.N = 𝐀*𝕟.N
𝕩.S = 𝐀*𝕟.S
```
The operator $*$ denotes matrix multiplication. 

<div class="alert alert-warning">
The net result is product of matrix 𝐀 and the concentration vector N.
</div>

<div class="alert alert-success">

**You have met the following learning objective:**
    
- Create and manipulate size distributions in code.
    
</div>

# 3. Representing DMAs

## 3.1 Representing DMA Geometry in Code

In [ ]:
play("Audio/dma-representation1.ogg")

The data type Λ defines the DMA in terms of flow rates and geometry and power supply polarity. The geometry parameters used in this example correspond to the TSI 3080 long column. The effective diffusion length describes diffusional loss in the DMA column (Reineking & Porstendörfer, 1986) which is described further below. Diffusional loss is ignored if leff = 0. Once instantiated Λ is queried to derive DMA/configuration specific properties.

In [ ]:
using DifferentialMobilityAnalyzers              # Load the package (only needed once)

qsa,qsh = 1.66e-5, 8.33e-5                       # Qsample [m3 s-1], Qsheath [m3 s-1]
t,p = 295.15, 1e5                                # Temperature [K], Pressure [Pa]
r₁,r₂,l = 9.37e-3,1.961e-2,0.44369               # DMA geometry [m]
leff = 13.0                                      # DMA effective diffusion length [m]
m = 6                                            # Upper number of charges
DMAtype = :cylindrical                           # specify DMA type as cylindrical or radial
polarity = :-                                    # negative :- or positive :+ polartiy

Λ = DMAconfig(t,p,qsa,qsh,r₁,r₂,l,leff,polarity,m,DMAtype)  

<div class="alert alert-info">
Write code in the cell below to instantiate a DMA with the following conditions: 
    
- positive polarity power supply
- room temperature
- Boulder pressure (840 hPa)
- cylindrical geometery
- no diffusional loss
- sheath-to-sample flow rate of 10:1
- sample flow rate 1 L min-1
- geometery of the TSI long column DMA
    
Name this DMA myΛ
</div>

## 3.2 Relationship between voltage, mobility, and diameter


In [ ]:
play("Audio/dma-representation2.ogg")

For the cylindrical DMA and balanced flows, the relationship between voltage, mobility, and diameter is given by Knutson and Whitby (1975)
<div class="alert alert-warning">
<center> $z^s = \frac{q_{sh}}{2\pi l v} \ln \left(\frac{r_2}{r_1}\right)$ </center>
where $v$ is the potential applied between the inner and out section of the annulus, $r_1$, $r_2$, and $l$ are the dimensions of the cylindrical DMA (Table 1) and $q_{sh}$ is the sheath flow rate. The relationship between diameter $d_p$ and centroid mobility "z star" ($z^s$) is 
<center> $d_p =  \frac{kec_c}{3\pi \eta z^s}$ </center>
where $e$ is the elementary charge and $k$ is the number of charges on the particle, $c_c$ is the Cunningham correction factor, and $\eta$ is the viscosity of the fluid. <br> 
</div>
    
Some of the conversions requires iteration. The function 

- ```dtoz(Λ,d)``` converts mobility diameter to mobility
- ```ztod(Λ,k,z)``` converts mobility to mobility diameter for particles carrying k charges
- ```vtoz(Λ, v)``` converts voltage to mobility
- ```ztov(Λ, z)``` converts mobility to voltage

and Λ subsumes the DMA geometery (r1,r2,l), sheath flow rate, and t,p.

In [ ]:
k = 1
v = 1000
z = vtoz(Λ, v)
dp = ztod(Λ, k, z)
@printf("V = %i V corresponds to Z = %.2e []\n", v, z)
@printf("Z = %.2e [] and n = %i charge(s) corresponds to Dp = %i nm\n", z, k, dp)
@printf("Dp = %i nm and n = 1 charge corresponds to Zp = %.2e nm\n", dp, dtoz(Λ,dp*1e-9))

<div class="alert alert-info">

Write code in the cell below to compute the mobility diameter of particles carrying two negative charges that would be selected at 5 kV potential by the DMA you configured above (myΛ).

</div>

<div class="alert alert-info">

Use the function ```dtoz(Λ,d)``` to verify that the diameter 114 nm corresponds to electrical mobility 2.20e-08 m2 V-1 s-1 for the DMA Λ, as claimed above. Note that d in dtoz is in units of m.

</div>

## 3.3 Representing the DMA Grid

In [ ]:
play("Audio/dma-representation3.ogg")

The DMA is operates between an upper and lower voltage limit. The full range is usually 10V to 10kV. At higher voltages the electric field breaks down. A convenient way to bin the DMA is to work with a log spaced mobility grid, which in essence is the size distribution histogram where diameters are computed using ztod. In code the DMA grid is represented using the DifferentialMobilityAnalyzer composite data type.

In [ ]:
bins,z₁,z₂ = 60, vtoz(Λ,10000), vtoz(Λ,10)    # bins, upper, lower mobility limit
δ = setupDMA(Λ, z₁, z₂, bins);                # Setup DMA grid

The ```δ``` structure that contains 

- ```Z``` are the mobility bin midpoints, 
- ```Dp``` are the diameter bin midpoints (internally in units of nm) 
- ```Ze``` are the mobility bin edges, 
- ```De``` are the diameter bin edges, 
- ```ΔlnD``` is the natural log ratio of upper and lower diameter bin edge

In [ ]:
# Above we generated 60 bins between 10V and 10 kV for DMA Λ. These are the midpoint diameter bins.
δ.Dp

<div class="alert alert-info">

Define your DMA grid myδ for your DMA myΛ, using a 30 bin representation and assuming that the DMA is operated between 50V and 5kV. Show the diameter grid.

</div>

<div class="alert alert-info">

Write a statement that shows the mobility grid corresponding to you diameter grid

</div>

<div class="alert alert-success">

**You have met the following learning objective**

- Represent cylindrical and radial DMAs with arbitrary dimensions in code.

</div>

## 3.4 (Diffusing) DMA Transfer Function

In [ ]:
play("Audio/transfer-function1.ogg")

In [ ]:
include("Scripts/plot_transfer_function1.jl")

**Figure.** Normalized DMA transfer function for DMA Λ with sheath-to-sample flow ratio β = 1/5  and 20 and 200 nm particles. The 20 nm transfer function is smeared due to diffusional broadening.

The DMA transfer function is the probability that a particle of a particle of a given size exits the classifier via the sample flow. The diffusive broadened DMA transfer function is computed assuming balanced sheath and excess flows using the expression of Stolzenburg and McMurry (2008).
<center> $ \Omega(\tilde{z},\beta,\sigma) = \frac{\sigma}{\sqrt{2}\beta}\left[\epsilon \left( \frac{\tilde{z}-(1+\beta)}{\sqrt{2}\sigma} \right) + \epsilon \left (\frac{\tilde{z}-(1-\beta)}{\sqrt{2}\sigma} \right) - 2\epsilon \left ( \frac{\tilde{z}-1}{\sqrt{2}\sigma}\right)  \right]$ </center>
    
where $\tilde{z} = \frac{z}{z^s}$ is the dimensionless mobility, $z$ is the particle mobility $z^s$ is the centroid mobility selected by the DMA, $\epsilon = x \mathrm{erf}(x) +\left(\exp(-x^2)/\sqrt{\pi}\right)$, $\mathrm{erf}$ is the error function, and $\beta = \frac{q_{sa}}{q_{sh}}$. The parameter $\sigma$ accounts for diffusional broading of the transfer function. Assuming plug flow, $\sigma$ can be computed using the following equations Hagwood (1999) <br> <br>
$\gamma = \left(\frac{r_1}{r_2}\right)^2$ <br>
$I = \frac{1}{2}(1+γ)$ <br>
$\kappa = \frac{lr_2}{r_2^2-r_1^2}$ <br>
$G = \frac{4(1+\beta)^2}{(1-γ)} \left[I+\{2(1+\beta)\kappa\}^{-2} \right ] $<br>
$\sigma = \sqrt{\frac{2G\pi ld_{ab}}{q_{sh}}}$ <br> <br>

In [ ]:
play("Audio/transfer-function2.ogg")

<div class="alert alert-warning">
The transfer function is included in the "DMA grid" δ

- ```δ.Ω(Λ,z,zˢ)``` where z is a vector of mobilities and zˢ ("z star") is the centroid mobility. 	
</div>

In [ ]:
zˢ = dtoz(Λ, 200e-9)      # centroid mobility for Dp = 200 nm
z = δ.Z                   # mobility grid defined for the DMA (60 bins above)
δ.Ω(Λ,z,zˢ)               # Output of the transfer function
DataFrame(z = z, Ω = δ.Ω(Λ,z,zˢ))  # To make a pretty table

In [ ]:
play("Audio/transfer-function3.ogg")

In [ ]:
include("Scripts/plot_transfer_function2.jl")

**Figure.** Example SMPS transfer function Ωav.

<div class="alert alert-warning">
    
During operation as scanning mobility particles, the voltage continuously changes. Signal is acquired during some discrete time interval $t_c$. The SMPS transfer function is calculated as the average transfer function during the time interval $[t,t+t_c]$ (Wang and Flagan, 1990). <br> <br>

<center> $\Omega_{av} = \frac{1}{tc}\int_{t_i}^{t_i+t_c} \Omega(Z,z^s(t)) dt$ </center>

where $t_i$ is the start time when counting begins in channel $i$, $z^s(t)$ is the selected centroid mobility at time t and is calculated from the applied voltage. The voltage is usually varied exponentially with time from high-to-low (downscan) or low-high (upscan). The average transfer function is obtained by numerical integration.
</div>

## 3.5 Equilibrium Charge Distribution Function

In [ ]:
play("Audio/charge-equilibrium.ogg")

In [ ]:
include("Scripts/plot_charge_fraction.jl")

Charging efficiency (charge equilibrium) obtained in the bipolar charger is computed based on the parameterized measurements by Wiedensohler et al. (1988) with coefficients taken from the TSI 3080 Manual (2009). 
<center> $T_c(k) = 10^\left\{ \sum_{i=1}^6 a_i (k) \left[ \ln \left(\frac{D_p}{nm}\right) \right]^{i-1} \right \}$ </center>
where $k = -2,-1,1,2$ is the number and polarity of particle charge and $a_i$ are empirical coefficients. <br> <br>
The function Tc(k) returns the charging efficiency for $k$ charges. Each function belongs to a DMA structure, which sets the internal diameter grid and polarity. In this script, the function δ.Tc is for negative polarity and δp.Tc for positive polarity as defined above. The function is computes charging efficiencies on the DMA internal grid δ.dp, which is set by the mobility range and the number bins.

For $k \ge \pm 3$, the formula from the TSI manual is used:
<center> $T_c(k) = \frac{e}{\sqrt{4\pi^2\epsilon D_pk_bT}} \exp \left( \frac{-\frac{\left[|k| - 2\pi\epsilon D_pk_bT \ln(0.875)\right]^2}{e^2}}{ \frac{4\pi\epsilon D_pk_bT}{e^2}} \right) $</center>
where $e$ is the elementary charge and $\epsilon$ is the dielectric constant for air.

<div class="alert alert-warning">
The charge efficiency function is included in the "DMA grid" δ

- ```δ.Tc(k,dp)``` where k is the number of charges and dp is the diameter in units of nm. 	
</div>

In [ ]:
# The fraction of charges carrying 1 charge. The polarity is defined by the DMA Λ that was used to 
# setup the grid δ 
δ.Tc(1,100.0) 

<div class="alert alert-info">

Write a one line expression that computes the fraction of particles carrying 3 charges  along the internal diameter grid of your DMA myδ.

</div>

<b> Figure. </b> Size dependence of fractional charging efficiency of through the bipolar charger for 1-4 charges.

## 3.6 Particle Losses/Detector Efficiency

In [ ]:
play("Audio/transmission-efficiency.ogg")

Some fraction of particles is lost during transit. Particles may not be detected at 100% efficiency. These losses can be described using a single loss function (or a series of loss functions). Here we consider just the penetration efficiency through the DMA as computed based on the parameterization of  Reineking & Porstendörfer (1986).

<div class="alert alert-warning">
The loss function is Tl included in the "DMA grid" δ

- ```δ.Tl(Λ,dp)``` where Λ is the DMA configuration (flow rate dependency and effective length) and dp is the diameter in units of nm. 	
</div>

In [ ]:
include("Scripts/plot_loss_fraction.jl")

<b> Figure. </b> Size dependence of penetration efficiency through the DMA for leff = 13 m and two flow rates.

<div class="alert alert-success">

**You have met the following learning objective**

- Review the theory governing DMA transmission.

</div>
